In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
train_data=pd.read_csv('data/train.csv')
test_data=pd.read_csv('data/test.csv')
print(test_data)

           id  feature_0  feature_1  feature_2  feature_3  feature_4  \
0      200000          0          0          0          0          0   
1      200001          1          2          0          0          0   
2      200002          0          1          7          1          0   
3      200003          0          0          0          4          3   
4      200004          0          0          5          0          0   
...       ...        ...        ...        ...        ...        ...   
99995  299995          0          0          0          0          0   
99996  299996          0          0          0          0          0   
99997  299997          0          0          0          0          0   
99998  299998          0          0          0          0          2   
99999  299999          0          0          0          0          0   

       feature_5  feature_6  feature_7  feature_8  ...  feature_65  \
0              0          0          0          0  ...           

In [3]:
features=train_data.drop('target',axis=1)
features=features.drop('id',axis=1)
target=train_data.target
target=target.replace("Class_","",regex=True)
target=torch.tensor(target.values.astype(np.int64))
target=target-1
# target.shape
features=torch.tensor(features.values,dtype=torch.float32)
train= torch.utils.data.TensorDataset(features,target)

trainloader=torch.utils.data.DataLoader(train,batch_size=64,shuffle=True)
print(features.shape)
# target = torch.tensor(target.values.astype(np.float64))

torch.Size([200000, 75])


In [4]:
from torch import nn
from torch import optim
model = nn.Sequential(nn.Linear(75, 36),
                      nn.ReLU(),
                      nn.Linear(36, 18),
                      nn.ReLU(),
                      nn.Linear(18,9),
                      nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.07)
epochs=10
for i in range(0,epochs):
    running_loss=0;
    for inputs, labels in trainloader:
#         images=images.view(images.shape[0],-1)
        out=model.forward(inputs)
#         print(out)
#         print(logits.shape)
#         print(labels.shape)
        loss=criterion(out,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
    print(running_loss/len(trainloader))


1.8209021158218384
1.7844460523223877
1.7779946694564819
1.7743639850997925
1.771988560523987
1.7703720263290406
1.7683500986862182
1.767326653175354
1.766260436592102
1.7651866329574586


In [5]:
accuracy=0
output_loss=0
for inputs,labels in trainloader:
    with torch.no_grad():
        test_out=model(inputs)
#         print(test_out)
        output_loss+=criterion(test_out,labels).item()
        logps=torch.exp(test_out)
#         print(logps)
        arr=(labels.data==logps.max(1)[1]).int()
        accuracy+=torch.sum(arr,dtype=torch.float)
print("Accuracy:",format(accuracy/features.shape[0]))
print("Loss:",format(output_loss/len(trainloader))) 

        
    



Accuracy: 0.35846999287605286
Loss: 1.7585711560058594


In [24]:
test_features=test_data.drop('id',axis=1)
test_features=torch.tensor(test_features.values,dtype=torch.float32)
# test_train=torch.utils.data.TensorDataset(test_features)
# testloader=torch.utils.data.DataLoader(test_train,batch_size=32,shuffle=True)
submission_file=pd.DataFrame(columns = ["Class_1 ","Class_2 ","Class_3 ","Class_4 ","Class_5 ","Class_6 ","Class_7 ","Class_8 ","Class_9 ",])
print(test_features.shape)

torch.Size([100000, 75])


In [25]:


for i in range(0,test_features.shape[0]):
    with torch.no_grad():
        test_out=model(test_features[i].view(1,75))
#         print(test_out)
#         output_loss+=criterion(test_out,labels).item()
        logps=torch.exp(test_out)
        logps=logps.numpy()
#         print(logps)
#         arr=(labels.data==logps.max(1)[1]).int()
#         accuracy+=torch.sum(arr,dtype=torch.float)
        submission_file.loc[len(submission_file.index)]=logps.reshape(9)
id1 = []
for i in range(200000,300000):
    id1.append(i)
submission_file.insert(0,'id',id1)
submission_file.to_csv('Hackathon-jun2.csv',index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'Hackathon-jun2.csv'